In [1]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/mlflow_project'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from mlflowProject.constants import *
from mlflowProject.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_inestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_inestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from mlflowProject import logger
from mlflowProject.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with folowing info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extact_zip_file(self):
        unzip_path = self.config.unzip_dir
        
        os.makedirs(unzip_path, exist_ok=True)
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extact_zip_file()

except Exception as e:
    raise e

[2024-03-15 15:47:35,231: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-15 15:47:35,237: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-15 15:47:35,238: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-15 15:47:35,239: INFO: common: created directory at: artifacts]
[2024-03-15 15:47:35,240: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-15 15:47:35,861: INFO: 3754641641: artifacts/data_ingestion/data.zip downloaded with folowing info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3CD2:E6A68:1DB0A5:1EAA9D:65F45F87
Accept-Ranges: bytes
Date: Fri, 15 